In [1]:
import numpy as np
import pandas as pd
from pyntcloud import PyntCloud
import os

In [2]:
def list_files_in_directory(directory_path):
    file_list = []
    for filename in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, filename)):
            file_list.append(filename)
    return file_list

In [3]:
directory = "abc-dataset-ply/"
files = sorted(list_files_in_directory("abc-dataset-ply/"))
dataset = []
def convert_to_binvox(path):
    point_cloud = np.loadtxt(path, skiprows=12)[:, 0:3]
    df = pd.DataFrame(data=point_cloud, columns=['x','y','z'])
    cloud = PyntCloud(df)
    voxelgrid_id = cloud.add_structure("voxelgrid", n_x=128, n_y=128, n_z=128)
    voxelgrid = cloud.structures[voxelgrid_id]
    Binary_voxel_array = voxelgrid.get_feature_vector(mode="binary")
    dataset.append(Binary_voxel_array)

In [4]:
for i in files:
    path = "abc-dataset-ply/" + i
    convert_to_binvox(path)

print(len(dataset))

21


In [5]:
train_data = dataset[:20]
test_data = dataset[20:]

In [6]:
import tensorflow as tf
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

2023-10-12 00:42:52.338726: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 00:42:57.256934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
original_dim = (128, 128, 128)
X_train = np.array(test_data, dtype=np.float32)

In [8]:
encoding_dim = 64

In [9]:
input_layer = Input(shape=original_dim + (1,))  # Add 1 channel dimension
x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling3D((2, 2, 2), padding='same')(x)

x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(encoded)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
decoded = Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(x)

2023-10-12 00:43:04.313884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 00:43:06.293335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 00:43:06.294100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
autoencoder = Model(inputs=input_layer, outputs=decoded)

In [11]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [12]:
autoencoder.fit(X_train, X_train, epochs=5, batch_size=2)

Epoch 1/5


2023-10-12 00:43:15.035410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-12 00:43:23.025684: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e201b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-12 00:43:23.025761: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2023-10-12 00:43:24.124084: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-12 00:43:24.959744: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/l

InternalError: Graph execution error:

Detected at node 'Adam/StatefulPartitionedCall_9' defined at (most recent call last):
    File "/home/hrishi/Miniconda3/envs/tensorflow/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/hrishi/Miniconda3/envs/tensorflow/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/hrishi/Miniconda3/envs/tensorflow/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/hrishi/Miniconda3/envs/tensorflow/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance
      app.start()
    File "/home/hrishi/Miniconda3/envs/tensorflow/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/hrishi/Miniconda3/envs/tensorflow/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
